In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import collections
import numpy as np
import PIL.Image as Image
import PIL.ImageColor as ImageColor
import PIL.ImageDraw as ImageDraw
import PIL.ImageFont as ImageFont
import six
import tensorflow as tf
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
PATH_TO_CKPT = '/home/evotianus/CarND-Capstone/training/v3.pb'
# List of the strings that is used to add correct label for each box.
# PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
PATH_TO_LABELS = "/home/evotianus/CarND-Capstone/training/labelmap_an.pbtxt"
# PATH_TO_LABELS = '/home/evotianus/Desktop/tensorflow_custom/models/object_detection/racoon/doggo-racoon.pbtxt'

NUM_CLASSES = 14

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
PATH_TO_TEST_IMAGES_DIR = 'test_images'
import glob
from random import shuffle
# TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(5, 7) ]
TEST_IMAGE_PATHS = glob.glob('/home/evotianus/CarND-Capstone/images/*.png')
shuffle(TEST_IMAGE_PATHS)
# TEST_IMAGE_PATHS = TEST_IMAGE_PATHS[0:50]
# Size, in inches, of the output images.
IMAGE_SIZE = (5, 5)

In [ ]:
TEST_IMAGE_PATHS = TEST_IMAGE_PATHS[0:100]

In [ ]:
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
# Each box represents a part of the image where a particular object was detected.
boxes_tensor = detection_graph.get_tensor_by_name('detection_boxes:0')
# Each score represent how level of confidence for each of the objects.
# Score is shown on the result image, together with the class label.
scores_tensor = detection_graph.get_tensor_by_name('detection_scores:0')
classes_tensor = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections_tensor = detection_graph.get_tensor_by_name('num_detections:0')

In [ ]:
import uuid
from datetime import datetime

timing = []
# unique_filename = 
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      start = datetime.now()
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes_tensor, scores_tensor, classes_tensor, num_detections_tensor],
          feed_dict={image_tensor: image_np_expanded})
      timing.append(datetime.now()-start)
      box = tuple(boxes[0].tolist())
      ymin, xmin, ymax, xmax = box[0]
      im_width, im_height = image.size
      (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
      left = int(left) +5
      right = int(right) - 5
      top = int(top) +5
      bottom = int(bottom) - 5
      img = image.crop((left,top,right,bottom))
      counted = collections.Counter(classes.tolist()[0])
      indx=int(counted.most_common()[0][0])
      pred = (categories[indx-1]['name'])
      pred_score = (scores[0][0])
      
#       img.save(fp="/home/evotianus/CarND-Capstone/training/V3/{}.png".format(str(uuid.uuid4())))